In [1]:
from bokeh.models import BoxZoomTool
import pandas as pd
import numpy as np

In [2]:
import glob
tics = glob.glob('/run/user/1000/gvfs/smb-share:server=silo.physics.usyd.edu.au,share=silo2/dhey3294/TESS/Searches/TESS_pg/res/*.png')

# Some glomming of files and ids
ticids = [tic.split('/res/')[-1].split('.')[0] for tic in tics]
tic_pg = pd.DataFrame({
    'ID': ticids,
    'files': tics
})

In [3]:
# from tqdm import tqdm

# # We need to chunk up our query because MAST will time-out.. sorry servers!
# chunked_ids = np.array_split(ticids, 100)
# df = Catalogs.query_criteria(catalog="Tic", ID=chunked_ids[0]).to_pandas()
# for chunked_id in tqdm(chunked_ids[1:]):
#     df = df.append(Catalogs.query_criteria(catalog="Tic", ID=chunked_id).to_pandas())

In [4]:
df = pd.read_csv('tic_query.csv', dtype={'ID': str})
# df.to_csv('tic_query.csv')

In [5]:
# # Let's merge our dataframes
# test = df.merge(tic_pg, left_on='ID', right_on='ID')
# mask = (np.isfinite(test['Teff'])) & (np.isfinite(test['lum']))

In [6]:
github_files = [f'https://raw.githubusercontent.com/danhey/CMDTess/master/data/{ticid}.png' for ticid in ticids]
tic_pg = pd.DataFrame({
    'ID': ticids,
    'files': github_files
})
test = df.merge(tic_pg, left_on='ID', right_on='ID')
mask = (np.isfinite(test['Teff'])) & (np.isfinite(test['lum']))

In [7]:
test['b-r'] = test['gaiabp'] - test['gaiarp']
test['absmag'] = test['GAIAmag'].values - 5 * np.log10(1/(test['plx'].values)*1000) + 5

/home/daniel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log10
  


In [17]:
plot_df = test[[
    'b-r',
    'absmag',
    'files',
    'ID',
    'GAIAmag'
]]

In [19]:
plot_df.to_csv('plot_df.csv')

In [8]:
from bokeh.transform import factor_cmap
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, ColorBar
from bokeh.palettes import Viridis
from bokeh.transform import linear_cmap
import bokeh

In [18]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool,  WheelZoomTool

output_file("index.html", title='TESS in the Southern Hemisphere')

source = ColumnDataSource(
        data=dict(
            x=plot_df['b-r'].values,
            y=plot_df['absmag'].values,
            imgs = plot_df['files'].values,
            desc = plot_df['ID'].values,
            color = plot_df['GAIAmag']
        )
    )

hover = HoverTool(# height="200" alt="@imgs" width="500"
        tooltips="""
        <div>
            <div>
                <img
                    src="@imgs"
                    border="2"
                ></img>
            </div>
            <div>
                <span style="font-size: 17px; font-weight: bold;">TIC @desc</span>
            </div>
        </div>
        """
    )

wheel = WheelZoomTool()
p = figure(#plot_width=900, plot_height=900, 
           tools=[hover, 'zoom_in', 'undo', wheel, 'reset', 'box_zoom'],
           title="TESS in the Southern Hemisphere", 
           toolbar_location="above",
           sizing_mode='stretch_both',
           x_range=(-1,6), y_range=(16,-5),
            output_backend="webgl"
          )

mapper = linear_cmap(field_name='color', palette=bokeh.palettes.Viridis256,low=min(plot_df['GAIAmag']) ,high=max(plot_df['GAIAmag']))


p.circle('x', 'y', 
         radius=0.001,
#          size=8,
         line_color=mapper,color=mapper, 
         fill_alpha=0.6,
         source=source)

p.xaxis.axis_label = 'Gaia BP - RP'
p.yaxis.axis_label = 'Gaia G absolute magnitude'
p.xaxis.axis_label_text_font_size = "18pt"
p.yaxis.axis_label_text_font_size = "18pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.title.text_font_size = "15pt"

p.xaxis.axis_label_text_font_style = 'normal'
p.yaxis.axis_label_text_font_style = 'normal'

p.toolbar.active_scroll=wheel

color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0))
p.add_layout(color_bar, 'right')

show(p)